In [1]:
import numpy as np
import pandas as pd
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import f1_score

In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format('../../Dataset/word2vec.bin', binary=True)

In [3]:
Local_data=pd.read_csv('../../Dataset/Local_LFT_dataset.csv')

In [4]:
Global_data=pd.read_csv('../../Dataset/global_LFT_dataset.csv')

In [5]:
def preprocessing(dataset):
    
    X, y = np.asarray(dataset['sentence']), np.asarray(dataset['global coarse grained'])
    label_map = {cat:index for index,cat in enumerate(np.unique(y))}
    y_prep = np.asarray([label_map[l] for l in y])
    x_tokenized = [[w for w in sentence.split(" ") if w != ""] for sentence in X]
   
    return x_tokenized, y_prep

In [8]:
def preprocessing_local(dataset):
    
    X, y = np.asarray(dataset['sentence']), np.asarray(dataset['coarse_grain'])
    label_map = {cat:index for index,cat in enumerate(np.unique(y))}
    y_prep = np.asarray([label_map[l] for l in y])
    x_tokenized = [[w for w in sentence.split(" ") if w != ""] for sentence in X]
   
    return x_tokenized, y_prep

In [9]:
class Sequencer():
    
    def __init__(self,
                 all_words,
                 max_words,
                 seq_len,
                 embedding_matrix
                ):
        
        self.seq_len = seq_len
        self.embed_matrix = embedding_matrix
        """
        temp_vocab = Vocab which has all the unique words
        self.vocab = Our last vocab which has only most used N words.
    
        """
        temp_vocab = list(set(all_words))
        self.vocab = []
        self.word_cnts = {}
        """
        Now we'll create a hash map (dict) which includes words and their occurencies
        """
        for word in temp_vocab:
            # 0 does not have a meaning, you can add the word to the list
            # or something different.
            count = len([0 for w in all_words if w == word])
            self.word_cnts[word] = count
            counts = list(self.word_cnts.values())
            indexes = list(range(len(counts)))
        
        # Now we'll sort counts and while sorting them also will sort indexes.
        # We'll use those indexes to find most used N word.
        cnt = 0
        while cnt + 1 != len(counts):
            cnt = 0
            for i in range(len(counts)-1):
                if counts[i] < counts[i+1]:
                    counts[i+1],counts[i] = counts[i],counts[i+1]
                    indexes[i],indexes[i+1] = indexes[i+1],indexes[i]
                else:
                    cnt += 1
        
        for ind in indexes[:max_words]:
            self.vocab.append(temp_vocab[ind])
                    
    def textToVector(self,text):
        # First we need to split the text into its tokens and learn the length
        # If length is shorter than the max len we'll add some spaces (100D vectors which has only zero values)
        # If it's longer than the max len we'll trim from the end.
        tokens = text.split()
        len_v = len(tokens)-1 if len(tokens) < self.seq_len else self.seq_len-1
        vec = []
        for tok in tokens[:len_v]:
            try:
                vec.append(self.embed_matrix[tok])
            except Exception as E:
                pass
        
        last_pieces = self.seq_len - len(vec)
        for i in range(last_pieces):
            vec.append(np.zeros(100,))
        
        return np.asarray(vec).flatten()

# 1. For Global LFT

In [10]:
x_tokenized, y_prep = preprocessing(Global_data)

In [11]:
Entities = np.asarray(Global_data['entity_name'])
e_tokenized = [[w for w in str(e).split(" ") if w != ""] for e in Entities]

In [12]:
entity_sequencer = Sequencer(all_words = [token for seq in e_tokenized for token in seq],
              max_words = 2000,
              seq_len = 15,
              embedding_matrix = model.wv
             )

/Users/Textory/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


In [13]:
e_vecs = np.asarray([entity_sequencer.textToVector(" ".join(seq)) for seq in e_tokenized])

In [14]:
from sklearn.decomposition import PCA

pca_model = PCA(n_components=700)
pca_model.fit(e_vecs)
print("Sum of variance ratios: ",sum(pca_model.explained_variance_ratio_))

Sum of variance ratios:  0.6619840127718994


In [15]:
e_comps = pca_model.transform(e_vecs)

In [16]:
x_train,x_test,y_train,y_test = train_test_split(e_comps,y_prep,test_size=0.2,random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(64104, 50)
(16026, 50)
(64104,)
(16026,)


In [17]:
svm_classifier = SVC()
svm_classifier.fit(x_train,y_train)

SVC()

In [18]:
y_pred = svm_classifier.predict(x_test)
micro = f1_score(y_test, y_pred, average="micro")
macro = f1_score(y_test, y_pred, average="macro")

In [20]:
print('micro_f1: ', micro)
print('macro_f1: ', macro)

micro_f1:  0.17889679271184325
macro_f1:  0.12690077825739185


# 2. For Local LFT

word2vec embeddings for the sentences

In [21]:
x_tokenized, y_prep = preprocessing_local(Local_data)

In [22]:
Entities = np.asarray(Local_data['entity_name'])
e_tokenized = [[w for w in str(e).split(" ") if w != ""] for e in Entities]

In [23]:
entity_sequencer = Sequencer(all_words = [token for seq in e_tokenized for token in seq],
              max_words = 2000,
              seq_len = 15,
              embedding_matrix = model.wv
             )

/Users/Textory/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


In [24]:
e_vecs = np.asarray([entity_sequencer.textToVector(" ".join(seq)) for seq in e_tokenized])

In [25]:
from sklearn.decomposition import PCA

pca_model = PCA(n_components=700)
pca_model.fit(e_vecs)
print("Sum of variance ratios: ",sum(pca_model.explained_variance_ratio_))

Sum of variance ratios:  0.9924208940574348


In [26]:
e_comps = pca_model.transform(e_vecs)

In [27]:
x_train,x_test,y_train,y_test = train_test_split(e_comps,y_prep,test_size=0.2,random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(82147, 700)
(20537, 700)
(82147,)
(20537,)


SVM model

In [ ]:
svm_classifier = SVC()
svm_classifier.fit(x_train,y_train)

In [19]:
y_pred = svm_classifier.predict(x_test)
micro = f1_score(y_test, y_pred, average="micro")
macro = f1_score(y_test, y_pred, average="macro")

0.17914638712092848

In [20]:
print('micro_f1: ', micro)
print('macro_f1: ', micro)